# Running Auto Correlation with Talon Deployer and BITE
###### Last Updated: 19/03/24

This demo will show the basic operation of auto correlation using the new Deployer and BITE TANGO devices currently deployed in this repository. With this notebook, all TANGO commands and attribute changes are made via a [TANGO DeviceProxy](https://pytango.readthedocs.io/en/stable/client_api/device_proxy.html) but the overall steps should be the same for using the JIVE interface or Taranta web interface.

## Pre-requisites


First, this notebook assumes you have a running environment launched from a pipeline, in particular it assumes you are running off one launched from the [SKA-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline. Secondly, for ease of dev work, it also assumes you are using a virtual env. This notebook was made with Python 3.10 in mind.

Finally, make sure all requirements are installed via [poetry]().

### Running on PSI


To Run on PSI, once running the launch step from the PSI pipeline, grab your booted namespace's name. You can run a check on the ns as well to make sure the pods are ready.

In [136]:
!kubectl get ns | grep ska-mid-psi

ci-ska-mid-psi-1212485592-jaredmda                       Active   5h27m
ci-ska-mid-psi-1212485592-jaredmda-sdp                   Active   5h28m
ci-ska-mid-psi-1222659231-alexschell                     Active   19m
ci-ska-mid-psi-1222659231-alexschell-sdp                 Active   20m
ci-ska-mid-psi-1224505247-maecst                         Active   3h21m
ci-ska-mid-psi-1224505247-maecst-sdp                     Active   3h21m


To make future work a bit easier we can load this into a NS var.

In [139]:
# Change this var to be the non-spd namespace with your username in it
NS = "ci-ska-mid-psi-1222659231-alexschell"

url = "https://142.73.34.170/"+NS+"/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run"
print("You can monitor board status using: https://142.73.34.170/{}/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run".format(NS))

You can monitor board status using: https://142.73.34.170/ci-ska-mid-psi-1222659231-alexschell/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run


We can then check to ensure the pods have spun up successfully.

In [138]:
!kubectl -n $NS get pods

NAME                                            READY   STATUS    RESTARTS   AGE
databaseds-ds-databaseds-tango-base-0           1/1     Running   0          18m
databaseds-tangodb-databaseds-tango-base-0      1/1     Running   0          18m
ds-bite-bite-0                                  1/1     Running   0          17m
ds-cbfcontroller-controller-0                   1/1     Running   0          16m
ds-cbfsubarray-cbfsubarray-01-0                 1/1     Running   0          16m
ds-cbfsubarray-cbfsubarray-02-0                 1/1     Running   0          16m
ds-cbfsubarray-cbfsubarray-03-0                 1/1     Running   0          16m
ds-deployer-deployer-0                          1/1     Running   0          17m
ds-fsp-fsp-01-0                                 1/1     Running   0          17m
ds-fsp-fsp-02-0                                 1/1     Running   0          17m
ds-fsp-fsp-03-0                                 1/1     Running   0          17m
ds-fsp-fsp-04-0             

## Initial Setup/Using Deployer Device

First we want to set up the target board that we will be using for the auto correlation. While this can be multiple boards, for now we will only need one. Punch out the board(s) you have access to and set the `TARGET_BOARD` to assign it to be used in future steps.

The `TANGO_HOST` will be created based of the namespace you set earlier.

In [140]:
import os
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Set to the boards you wish to configure.
TARGET_BOARD = [1]


TANGO_HOST = "databaseds-tango-base."+NS+".svc.cluster.local:10000"
print("Will be using HOST: ", TANGO_HOST)

os.environ["TANGO_HOST"] = TANGO_HOST

Will be using HOST:  databaseds-tango-base.ci-ska-mid-psi-1222659231-alexschell.svc.cluster.local:10000


We then load in the locations of local files for configuration. If you wish to use custom files you can change the folders pointed to, otherwise this code block can be run without edits.

In [141]:
# Get files from configs folder in repo
currdir = os.path.dirname(os.getcwd())
correlator_file_location = os.path.abspath(os.path.join(currdir,"configs/auto-correlation"))
common_file_location = os.path.abspath(os.path.join(currdir,"configs/common"))

print(os.listdir(correlator_file_location))
print(os.listdir(common_file_location))

['assign_resources.json']
['hw_config', 'slim_config', 'sys_params']


Next, we pass in the SLIM mesh config files by copying them to the namespace, these files should be in the json_files storage folder. Custom files here are not required, but if needed, the following two code blocks can be used.

In [142]:
# Change this value if you want to specify a certain slim directory for files
slim_fs_config = ""
slim_vis_config = ""
if slim_fs_config != "" and slim_vis_config != "" :
    fs_config_path = os.path.join(correlator_file_location, slim_fs_config)
    vis_config_path = os.path.join(correlator_file_location, slim_vis_config)    
else:
    print("SLIM will use defaults for this test.")

SLIM will use defaults for this test.


In [ ]:
!"kubectl cp {fs_config_path} {NS}/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml"
!"kubectl cp {vis_config_path} {NS}/ds-cbfcontroller-controller-0:/app/mnt/slim/vis_slim_config.yaml"

For this demo, we will interact with the TANGO devices via a device proxy, which will allow us to pass commands into them as we would in the UI.

In [143]:
from PyTango import DeviceProxy, Database

# Setup the device proxies targeting bite and deployer
db = Database()
deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")

#Check the devices initially deployed to the database
devices = db.get_device_exported("*")
print(devices)
# Make sure the deployer device is set to ON
deployer_tango.On()
print("State of deployer device:")
print(deployer_tango.state())

DbDatum[
        name = '*'
value_string = ['dserver/CbfController/controller', 'dserver/CbfSubarray/cbfsubarray-01', 'dserver/CbfSubarray/cbfsubarray-02', 'dserver/CbfSubarray/cbfsubarray-03', 'dserver/DataBaseds/2', 'dserver/ECBite/bite', 'dserver/ECDeployer/deployer', 'dserver/FspMulti/fsp-01', 'dserver/FspMulti/fsp-02', 'dserver/FspMulti/fsp-03', 'dserver/FspMulti/fsp-04', 'dserver/MidCspCapabilityFsp/capabilityfsp', 'dserver/MidCspCapabilityVcc/capabilityvcc', 'dserver/MidCspController/controller', 'dserver/MidCspSubarray/subarray1', 'dserver/MidCspSubarray/subarray2', 'dserver/MidCspSubarray/subarray3', 'dserver/PowerSwitch/powerswitch-001', 'dserver/PowerSwitch/powerswitch-002', 'dserver/PowerSwitch/powerswitch-003', 'dserver/PowerSwitch/powerswitch-004', 'dserver/Slim/mesh', 'dserver/SlimLink/fs-links', 'dserver/SlimLink/vis-links', 'dserver/TalonBoard/talon-001', 'dserver/TalonBoard/talon-002', 'dserver/TalonBoard/talon-003', 'dserver/TalonBoard/talon-004', 'dserver/TalonBoard

First, we set the target talon boards we want to set up our configuration for. In Jive/Taranta, this would be configured by manually writing the attribute via the UI. Multiple boards can be targeted.

In [145]:
deployer_tango.write_attribute("targetTalons",TARGET_BOARD)

With this set, we can then run the configuration command by calling the generate_config_jsons command.

In [146]:
deployer_tango.command_inout("generate_config_jsons")

We then get the device artifacts from the [artifact repository](https://artefact.skao.int/#browse/browse:helm-internal) by running the command via TANGO. This step may take some time as it downloads multiple devices.

In [147]:
deployer_tango.set_timeout_millis(200000)
try:
    deployer_tango.command_inout("download_artifacts")
except Exception as e:
    print(e)
    print("Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes.")
deployer_tango.set_timeout_millis(4500)

To check that the artifacts downloaded successfully, we want to check that the following returns something like `INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading`. 

In [148]:
!kubectl logs -n $NS ds-deployer-deployer-0 -f | grep 'Finished downloading'

1|2024-03-23T00:22:46.140Z|INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading
^C


Finally, we can configure the TANGO database with all the tango devices we just downloaded using the ConfigDB command.

In [149]:
deployer_tango.command_inout("configure_db")

The TANGO database should now be configured with all the devices needed for the next step, running the BITE device.

In [150]:
print(*db.get_device_exported("*").value_string, sep="\n")

dserver/CbfController/controller
dserver/CbfSubarray/cbfsubarray-01
dserver/CbfSubarray/cbfsubarray-02
dserver/CbfSubarray/cbfsubarray-03
dserver/DataBaseds/2
dserver/ECBite/bite
dserver/ECDeployer/deployer
dserver/FspMulti/fsp-01
dserver/FspMulti/fsp-02
dserver/FspMulti/fsp-03
dserver/FspMulti/fsp-04
dserver/MidCspCapabilityFsp/capabilityfsp
dserver/MidCspCapabilityVcc/capabilityvcc
dserver/MidCspController/controller
dserver/MidCspSubarray/subarray1
dserver/MidCspSubarray/subarray2
dserver/MidCspSubarray/subarray3
dserver/PowerSwitch/powerswitch-001
dserver/PowerSwitch/powerswitch-002
dserver/PowerSwitch/powerswitch-003
dserver/PowerSwitch/powerswitch-004
dserver/Slim/mesh
dserver/SlimLink/fs-links
dserver/SlimLink/vis-links
dserver/TalonBoard/talon-001
dserver/TalonBoard/talon-002
dserver/TalonBoard/talon-003
dserver/TalonBoard/talon-004
dserver/TalonBoard/talon-005
dserver/TalonBoard/talon-006
dserver/TalonBoard/talon-007
dserver/TalonBoard/talon-008
dserver/TalonDxLogConsumer/001


## Setting Controller settings

Now that we have the controller device deployed, we can use it to prep for further steps.

In [151]:
CBF_tango = DeviceProxy("mid_csp_cbf/sub_elt/controller")
control_tango = DeviceProxy("mid-csp/control/0")
subarray_tango = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")
print(CBF_tango.status())

The device is in DISABLE state.


We can then set adminMode to 0 (ONLINE), allowing us to run commands, and set simulationMode to 0 (FALSE).

In [158]:
CBF_tango.write_attribute("adminMode",0)
CBF_tango.write_attribute("simulationMode",0)
subarray_tango.write_attribute("adminMode",0)
subarray_tango.write_attribute("simulationMode",0)
control_tango.write_attribute("adminMode",0)
control_tango.write_attribute("simulationMode",0)
control_tango.write_attribute("cbfSimulationMode",0)
print(CBF_tango.State())
print(control_tango.State())

if CBF_tango.read_attribute("adminMode").value == 0 and CBF_tango.read_attribute("simulationmode").value == 0:
    print ("Set values successfully!")
elif CBF_tango.read_attribute("adminMode").value == 0 and CBF_tango.read_attribute("simulationmode").value == 1:
    print ("Set to simulation mode on.")
else:
    print ("Error, couldn't set values!")


OFF
OFF
Set values successfully!


Next, we load in a inital values parameters and pass it to the controller, to do this we read in JSON file and pass it as a DevString to the relevant device command:

In [161]:

init_sys_param_file = os.path.join(common_file_location, "sys_params/initial_system_param.json")

with open(init_sys_param_file) as init_file:
    data = init_file.read()
# Use the InitSysParam command and feed in data
    
# add in check for off state before writing
CBF_tango.command_inout("InitSysParam", data)
print(subarray_tango.sysParam)


CBF_tango.Init()
CBF_tango.On()
control_tango.Init()
control_tango.On()
print(CBF_tango.status())

{
	"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0",
	"dish_parameters": {
		"SKA001": {
			"vcc": 1,
			"k"	 : 100
		},
		"SKA036": {
			"vcc": 2,
			"k"	 : 618
		},
		"SKA063": {
			"vcc": 3,
			"k"	 : 1220
		},
		"SKA100": {
			"vcc": 4,
			"k"	 : 1815
		}
	}
}


DevFailed: DevFailed[
DevError[
    desc = Command On not allowed when the device is in DISABLE state
  origin = CORBA::Any* Tango::DeviceClass::command_handler(Tango::DeviceImpl*, const string&, const CORBA::Any&) at (/src/cppTango/cppapi/server/deviceclass.cpp:1139)
  reason = API_CommandNotAllowed
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device mid-csp/control/0, command On
  origin = virtual Tango::DeviceData Tango::Connection::command_inout(const string&, const Tango::DeviceData&) at (/src/cppTango/src/client/devapi_base.cpp:1334)
  reason = API_CommandFailed
severity = ERR]
]

TODO: Add in HW swap for using higher # Talon boards.

## Running Commands Through the BITE Device

Now that the BITE tango Device has been deployed via the deployer, we can use it to configure tests. First we check the device is running.

In [128]:
target_mac_address = "08:c0:eb:9d:47:78"

bite_tango = DeviceProxy("mid_csp_cbf/ec/bite")
# Running this should return RUNNING
print(bite_tango.State())

RUNNING


Now that we have confirmed it is running, we load in what board we want to configure.

In [129]:
bite_tango.write_attribute("boards",TARGET_BOARD)
bite_tango.write_attribute("bite_mac_address", target_mac_address)

For now, we can use the defaults and simply call the write command for the test configs. This should return the configuration for each board passed in.

In [133]:
#print(bite_tango.command_inout("generate_bite_data"))

# For now run the backup
!kubectl exec -ti -n $NS ec-bite -- python3 midcbf_bite.py --talon-bite-config --boards=1 --bite_mac_address=$target_mac_address


[midcbf_bite.py: line 43]INFO: User: root
[midcbf_bite.py: line 118]INFO: Test parameter files have not all been copied to, or are not all accessible at, their expected location. Cannot proceed with specified test parameters unless all files are present, since some files would be pointing to non-existent information expected in other files. Resorting to the default test parameters in basic_test_parameters.json.
[midcbf_bite.py: line 172]INFO: No test parameter arguments provided. Defaulting to basic test parameters on boards: ['1']
[midcbf_bite.py: line 182]INFO: BITE receptors: [{'talon': '1', 'dish_id': 'SKA001', 'sample_rate_k': 100, 'bite_config_id': 'basic gaussian noise', 'bite_initial_timestamp_time_offset': 60.0}]
[midcbf_bite.py: line 203]INFO: Talon BITE Configure
[midcbf_bite.py: line 118]INFO: Creating dps for device server: dsgaussiannoisegen
[midcbf_bite.py: line 680]INFO: Creating device server for talondx-001/gaussiannoisegen/gn_gen_src_polX_0
[midcbf_bite.py: line 660]

In [ ]:
!kubectl -n $NS logs ds-bite-bite-0 | grep generate_bite_data

With the bite data generated, we can then start generating LSTV playback data:

In [ ]:
bite_tango.command_inout("start_lstv_replay")


We can then check the logs to ensure the LSTV geneartion runs correctly.

In [135]:
!kubectl -n $NS logs ds-bite-bite-0 -f

[THREAD: MainThread]->[bite.py: line 192]INFO: Start TANGO device init...
1|2024-03-22T23:58:12.538Z|INFO|MainThread|write_loggingLevel|base_device.py#1085|tango-device:mid_csp_cbf/ec/bite|Logging level set to LoggingLevel.INFO on Python and Tango loggers
1|2024-03-22T23:58:12.538Z|INFO|MainThread|update_logging_handlers|base_device.py#313|tango-device:mid_csp_cbf/ec/bite|Logging targets set to ['tango::logger']
1|2024-03-22T23:58:12.545Z|INFO|MainThread|exit|core.py#136||Finished processing state UNINITIALISED exit callbacks.
1|2024-03-22T23:58:12.545Z|INFO|MainThread|enter|core.py#130||Finished processing state INIT enter callbacks.
1|2024-03-22T23:58:12.545Z|INFO|MainThread|_update_state|base_device.py#905|tango-device:mid_csp_cbf/ec/bite|Device state changed from UNKNOWN to INIT
1|2024-03-22T23:58:12.546Z|INFO|MainThread|callbacks|core.py#1147||Executed callback '<bound method OperationStateMachine._state_changed of <ska_tango_base.state_machine.OperationStateMachine object at 0x7f

## Assigning Resources

Next, we use the relevant subarray device to assign resources. First, as with the other devices, we establish a DeviceProxy to connect to it. We also read in the assign_resources file to load in using the command. 

In [ ]:
subarray_tango = DeviceProxy("mid-csp/subarray/01")
print(subarray_tango.State())
assign_resources_file = os.path.join(correlator_file_location, "assign_resources.json")
with open(init_sys_param_file) as init_file:
    resource_data = init_file.read()

subarray_tango.On()
print(subarray_tango.State())

Next,we run the actual command to pass in the resources:

In [ ]:
subarray_tango.command_inout("AssignResources",resource_data)

## SDP Setup

Next, we run a few commands to set up the Science Data Processor (SDP), utilizing the sdp namespace

In [ ]:
sdp_namespace = NS+"-sdp"
print(sdp_namespace)
!kubectl -n $sdp_namespace get pods

Now, we assign resources as we did with the subarray: